In [1]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName('Employee Analysis').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/19 14:43:50 WARN Utils: Your hostname, Pulastyas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.40 instead (on interface en0)
26/01/19 14:43:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 14:43:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
emp = spark.read.options(header=True, inferSchema=True).csv('data/input/db_employee.csv')

emp.show()

+-----+----------+---------+------+-------------+
|   id|first_name|last_name|salary|department_id|
+-----+----------+---------+------+-------------+
|10301|     Keith|   Morgan| 27056|            2|
|10302|     Tyler|    Booth| 32199|            3|
|10303|  Clifford|   Nguyen| 32165|            2|
|10304|      Mary|    Jones| 49488|            3|
|10305|   Melissa|   Lucero| 27024|            3|
|10306|    Ashley|       Li| 28516|            4|
|10307|    Joseph|  Solomon| 19945|            1|
|10308|   Anthony|  Sanchez| 43801|            3|
|10309| Katherine|  Huffman| 12984|            4|
|10310|      Dawn|    Foley| 28902|            2|
|10311|   Melissa|   Holmes| 33575|            1|
|10312|  Kathleen|    Davis| 44579|            2|
|10313|      Adam|  Simmons| 15442|            6|
|10314|   Desiree|   Brewer| 37494|            6|
|10315|    Sierra| Anderson| 20592|            6|
|10316|      Beth|   Torres| 34902|            1|
|10317|    Pamela|Rodriguez| 48187|            4|


In [4]:
dept = spark.read.options(header=True, inferSchema=True).csv('data/input/db_department.csv')

dept.show()

+---+--------------+
| id|    department|
+---+--------------+
|  1|   engineering|
|  2|human resource|
|  3|     operation|
|  4|     marketing|
|  5|         sales|
|  6| customer care|
+---+--------------+



In [22]:
max_sal = emp.join(dept, on=emp.department_id == dept.id, how="inner")\
    .filter(F.col('department').isin(['marketing', 'engineering']))\
    .groupBy()\
    .pivot('department')\
    .agg(F.max('salary').alias('max_sal'))\
    .withColumn('salary_difference', F.abs(F.col('marketing') - F.col('engineering')))\
    .select('salary_difference')

max_sal.show()

+-----------------+
|salary_difference|
+-----------------+
|             2400|
+-----------------+



In [23]:
spark.stop()